In [ ]:
import os
import glob

# Third-party
from astropy.io import fits
import astropy.coordinates as coord
from astropy.stats import LombScargle
import astropy.units as u

import matplotlib as mpl
import matplotlib.pyplot as pl
from matplotlib.gridspec import GridSpec
import numpy as np
pl.style.use('apw-notebook')
%matplotlib inline
from scipy.signal import argrelmax

In [ ]:
bright_files = []
files = glob.glob("../data/light_curves/*.fits")
for filename in files:
    header = fits.getheader(os.path.abspath(filename), 0)
    Kp = header['KEPMAG']
    
    if Kp < 15:
        bright_files.append(filename)

In [ ]:
def plot_periodogram(filename, n_terms=2):
    n_fit_freq = 3 # can't change this right now because of gridspec
    
    data = fits.getdata(os.path.abspath(filename), 1)
    still = data['MOVING'] != 1.
    t = data['T'][still]*u.day
    y = data['FCOR'][still]
    
    ls = LombScargle(t, y, nterms=n_terms)
    
    freq,power = ls.autopower(minimum_frequency=1/(1.2*u.day), 
                              maximum_frequency=1/(0.2*u.day))

    fig = pl.figure(figsize=(15,15))

    gs = GridSpec(3,3)

    ax = fig.add_subplot(gs[0,0])
    ax.plot(t-t.min(), y, linestyle='none')
    ax.set_xlabel("Time [day]")
    ax.set_ylabel("Flux")

    ax = fig.add_subplot(gs[1,0])
    ax.plot(1/freq, power)
    ax.set_xlabel("Period [day]")
    ax.set_ylabel("Power")

    # best freqs
    idx, = argrelmax(power)
    idx = sorted(idx, key=lambda x: power[x], reverse=True)[:n_fit_freq]

    # compute the best-fit model
    phase = np.linspace(0, 1)
    for i in range(n_fit_freq): # n best periods
        fit_freq = freq[idx[i]]
        fit_power = power[idx[i]]
        mag_fit = ls.model(t=phase / fit_freq, frequency=fit_freq)

        ax = fig.add_subplot(gs[i,1:])
        ax.plot((t*fit_freq) % 1, y, linestyle='none')
        ax.plot(phase, mag_fit, marker=None)
    
        ax.text(0.75, y.min() + (y.max()-y.min())*0.05, fontsize=24,
                s="power: {:.3f}".format(fit_power))
    
        if i < 2:
            ax.xaxis.set_ticklabels([""])

    ax.set_xlabel("Phase")
    
    fig.tight_layout()

In [ ]:
for filename in bright_files[4:8]:
    plot_periodogram(filename, 4)

---

In [ ]:
from gala.util import rolling_window

In [ ]:
def moving_best_periods(filename, n_terms=2):
    data = fits.getdata(os.path.abspath(filename), 1)
    still = data['MOVING'] != 1.
    t = data['T'][still]*u.day
    y = data['FCOR'][still]
    
    n_window = int(4 * (0.5*u.day) / np.median(t[1:] - t[:-1]))
    stride = n_window // 8
    
    best_freqs = []
    for (i1,i2),t_window in rolling_window(t, n_window, stride=stride, return_idx=True):
        ls = LombScargle(t_window, y[i1:i2], nterms=n_terms)    
        freq,power = ls.autopower(minimum_frequency=1/(1.2*u.day), 
                                  maximum_frequency=1/(0.2*u.day),
                                  samples_per_peak=10)
        best_freq = freq[power.argmax()]
        best_freqs.append(best_freq)
        
    return 1 / u.Quantity(best_freqs)

In [ ]:
best_periods = moving_best_periods(files[5])

In [ ]:
pl.plot(best_periods)